This notebook is part of Andreu's (esdandreu@gmail.com) Master Thesis work at
Keio University.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AcousticOdometry/AO/blob/main/notebooks/train_model.ipynb)


# Setup

This section will take care of installing the necessary packages as well as
configuring some environment variables.


## Colab

Assess wether the notebook is being executed in [Google
Colab](https://colab.research.google.com/) and if so, set up the software
needed in Colab runtime.


In [ ]:
%%capture
try:
    from google import colab
    COLAB_RUNTIME = True
    colab.drive.mount('/content/drive')
except ImportError:
    COLAB_RUNTIME = False

## Packages


In [ ]:
import logging

from typing import Dict
from pathlib import Path
from datetime import datetime
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary
from torchvision import datasets, transforms

### AO

Setup Acoustic Odometry python package. If this notebook is being executed in
[Colab](#colab), the package will be installed from Github. Because of this, a
Github [personal access
token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
is needed for the installation.

If the notebook is not running on Colab and the package is not already
installed, installation instructions will be prompted.


In [ ]:
if COLAB_RUNTIME:
    import subprocess
    import requests
    import sys
    import os
    #@markdown Use a [GitHub Personal Access Token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
    GITHUB_TOKEN = ''  #@param {type:"string"}
    auth = requests.auth.HTTPBasicAuth('', GITHUB_TOKEN)
    response = requests.get(
        "https://api.github.com/repos/AcousticOdometry/AO/releases/latest",
        auth=auth
        )
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        raise RuntimeError(
            'Check GITHUB_TOKEN is a Personal Access Token with repo access'
            )
    headers = {'Accept': 'application/octet-stream'}
    for asset in response.json()['assets']:
        r = requests.get(
            asset['url'], auth=auth, allow_redirects=True, headers=headers
            )
        r.raise_for_status()
        wheel_name = asset['name']
        with open(wheel_name, 'wb') as f:
            f.write(r.content)
        try:
            result = subprocess.check_output([
                sys.executable, '-m', 'pip', 'install', wheel_name
                ])
            print(f'Installed {wheel_name}')
            break
        except subprocess.CalledProcessError as e:
            pass
        finally:
            os.remove(wheel_name)
    import ao
else:
    try:
        import ao
    except ImportError:
        raise ImportError(
            "Acoustic Odometry python extension is not installed. Check "
            r"https://github.com/AcousticOdometry/AO#readme"
            " for detailed instructions."
            )

## Configure logging

In [5]:
logging_folder = Path().parent / 'logs'
logging_folder.mkdir(exist_ok=True)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(
            logging_folder / datetime.now().strftime('%Y%m%d_%H%M%S.log')
            ),
        stream_handler,
        ],
    level=logging.DEBUG
    )


# Training

## Define the model

In [ ]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(105408, 300)
        self.fc2 = nn.Linear(300, len(dataset.classes))


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  

## Load data

In [17]:
if COLAB_RUNTIME:
    # @markdown Check where is the experiment folder situated in your drive folder.
    # @markdown Remember that if you have been shared the folder, you can
    # @markdown [add a shortcut to your drive](https://support.google.com/drive/answer/9700156?hl=en&co=GENIE.Platform%3DDesktop)
    # @markdown in order to make it available in google colab.
    experiment = "/content/drive/MyDrive/VAO_WheelTestBed-Experiment-1"  #@param {type:"string"}
    EXPERIMENT_FOLDER = Path(experiment)
    if not EXPERIMENT_FOLDER.is_dir():
        raise RuntimeError(f'Invalid experiment folder {EXPERIMENT_FOLDER}')
else:
    EXPERIMENT_FOLDER = ao.dataset.utils.get_folder(
        env='WHEELTESTBED_EXPERIMENT1'
        )
DATASETS_FOLDER = EXPERIMENT_FOLDER / 'datasets'
MODELS_FOLDER = EXPERIMENT_FOLDER.parent.parent / 'models'
MODELS_FOLDER.mkdir(exist_ok=True)

In [ ]:
def load_datasets(
    dataset_folder: Path,
    train_split: float = 0.8,
    batch_size: int = 15,
    validation_dataset_folder: Path = None,
    ) -> Dict[str, torch.utils.data.DataLoader]:
    pass

In [7]:
dataset_folder = DATASETS_FOLDER / 'initial_cnn_samples'
dataset = datasets.ImageFolder(
    root=dataset_folder,
    transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
        ])
    )
print(dataset.class_to_idx)

{'000': 0, '001': 1, '002': 2, '003': 3, '004': 4, '005': 5, '006': 6}


In [8]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size]
    )

print("Training size:", len(train_dataset))
print("Testing size:", len(test_dataset))


Training size: 32158
Testing size: 8040


In [10]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [12]:
model = CNNet().to(device)

In [13]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    logging.info(f'acc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}')


def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    # TODO use tqdm
    for batch_n, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch_n % 100 == 0:
            loss, current = loss.item(), batch_n * len(X)
            logging.debug(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


In [ ]:
def train_and_test(dataset, model, epochs, validation_dataset=None):
    model.train()
    # size = len(dataloader.dataset)
    # TODO dataset into dataloaders
    # TODO epochs
    # TODO use tqdm
    for batch_n, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch_n % 100 == 0:
            loss, current = loss.item(), batch_n * len(X)
            logging.debug(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

In [14]:
epochs = 15

for t in range(epochs):
    logging.info(f'Epoch {t}')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)


[INFO] Epoch 0
[INFO] 
Test Error:
acc: 66.2%, avg loss: 0.063582

[INFO] Epoch 1
[INFO] 
Test Error:
acc: 75.7%, avg loss: 0.049014

[INFO] Epoch 2
[INFO] 
Test Error:
acc: 81.4%, avg loss: 0.037889

[INFO] Epoch 3
[INFO] 
Test Error:
acc: 84.1%, avg loss: 0.032734

[INFO] Epoch 4
[INFO] 
Test Error:
acc: 83.7%, avg loss: 0.031658

[INFO] Epoch 5
[INFO] 
Test Error:
acc: 86.5%, avg loss: 0.027330

[INFO] Epoch 6
[INFO] 
Test Error:
acc: 87.2%, avg loss: 0.025854

[INFO] Epoch 7
[INFO] 
Test Error:
acc: 88.0%, avg loss: 0.024643

[INFO] Epoch 8
[INFO] 
Test Error:
acc: 88.9%, avg loss: 0.022725

[INFO] Epoch 9
[INFO] 
Test Error:
acc: 89.3%, avg loss: 0.022265

[INFO] Epoch 10
[INFO] 
Test Error:
acc: 89.6%, avg loss: 0.021928

[INFO] Epoch 11
[INFO] 
Test Error:
acc: 89.9%, avg loss: 0.021545

[INFO] Epoch 12
[INFO] 
Test Error:
acc: 90.4%, avg loss: 0.020569

[INFO] Epoch 13
[INFO] 
Test Error:
acc: 90.4%, avg loss: 0.020617

[INFO] Epoch 14
[INFO] 
Test Error:
acc: 90.1%, avg loss: 

In [18]:
summary(model, input_size=(15, 3, 256, 120))
torch.save(model, MODELS_FOLDER / 'initial_cnn_samples.pt')